<a href="https://colab.research.google.com/github/shivamsinghtomar78/LangChain/blob/main/tools_in_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# First, uninstall any existing versions to avoid conflicts
!pip uninstall -y ddgs duckduckgo-search

# Install the required packages with specific versions that work together
!pip install -U ddgs duckduckgo-search

# Now install langchain components
!pip install langchain langchain-core langchain-community pydantic langchain_experimental


Found existing installation: duckduckgo_search 8.1.1
Uninstalling duckduckgo_search-8.1.1:
  Successfully uninstalled duckduckgo_search-8.1.1
  Using cached duckduckgo_search-8.1.1-py3-none-any.whl.metadata (16 kB)
Using cached duckduckgo_search-8.1.1-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 64.2 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


Built-in Tool - DuckDuckGo Search

In [12]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('check this company is authentic Labmentix in english ')

print(results)

At Labmentix , we solve a critical problem: the lack of real -world experience that keeps talented individuals from landing their dream jobs. Website: labmentix .in. External link for Labmentix Private Limited ; Industry: IT Services and IT Consulting ; Company size: 51-200 employees ; Headquarters ... We build Intensive Programs in Collaboration With Industry Experts to give Real Work Experience and Help You Land a Job. labmentix .in. Jul 31, 2025 — Bangalore-based startup Labmentix is transforming how India's aspiring IT professionals gain job-ready skills. Founded by Swaraj Phatangade in January 2025, ... Jun 13, 2025 · But it’s super important to verify before you apply — especially when the offer seems too easy, quick, or flashy. If Labmentix asks for money, run the other way. Always do …


In [13]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


Built-in Tool - Shell Tool

In [14]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('mkdir shivam')

print(results)

Executing command:
 mkdir shivam



/usr/local/lib/python3.12/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Custom Tools

In [17]:
from langchain_core.tools import tool

In [18]:
#step 1-create a function
def multiply(a,b):
  """multiply two numbers"""
  return a*b

In [19]:
#step 2 -add type hints
def multiply(a:int,b:int)->int:
  """multiply two numbers"""
  return a*b

In [20]:
#step 3-add tools decorator
@tool
def multiply(a:int,b:int)->int:
  """multiply two numbers"""
  return a*b

In [21]:
result = multiply.invoke({"a":3, "b":5})

In [22]:
print(result)

15


In [23]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [24]:
print(multiply.args_schema.model_json_schema())

{'description': 'multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


Method 2 - Using StructuredTool

In [25]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [26]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [27]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [28]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [29]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


Method 3 - Using BaseTool Class

In [30]:
from langchain.tools import BaseTool
from typing import Type

In [31]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [32]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [33]:
multiply_tool = MultiplyTool()

In [34]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


Toolkit

In [35]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [36]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [37]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
